# Learnability Project: PFA Phonotactic Learner

------

## General functions and other stuff

In [123]:
from pyfoma import FST, State
import copy

Lfish = "⋊"
Rfish = "⋉"


def data_iterator(path):
    """Yield a word as a list of tokens"""
    with open (path, "r") as fin:
        for line in fin.readlines():
            yield line.split()
            
            
def make_alphabet(path) -> list:
    """This function extracts an alphabet from a corpusl"""
    alph = []
    for word in data_iterator(path):
        for sym in word:
            if sym not in alph:
                alph.append(sym)
    return alph

## Strictly 2-Local Functions

In [ ]:
def make_2_SL_dfa(alphabet:list) -> FST:
    """This function initializes a 2-SL PFA according to an alphabet. All weights are set to 0 by default"""
    
    # create the FST object and instantiate the initial state
    dfa = FST()
    q0 = dfa.initialstate
    q0.finalweight = 0
    q0.name = Lfish
    states = {q0,}
    
    # initialize all states and connect q0 to them
    for sym in alphabet:
        q = State()
        q.name = sym
        q0.add_transition(q, sym, 0)
        states.add(q)
        q.finalweight = 0
    
    # loopify the states, excluding q0
    states_no_q0 = set([s for s in states if s.name != Lfish])
    for state1 in states_no_q0:
        for state2 in states_no_q0:
            state1.add_transition(state2, state2.name, 0)
    
    dfa.states      = states
    dfa.finalstates = states
    dfa.alphabet    = alphabet
    return dfa

def update_SL(dfa:FST, path:str) -> FST:
    """
    This function takes a 2SL dfa and updates each transition whenever it is "passed through".
    
    More specifically, it iterates over the outgoing transitions for each state, checks which transition label
    matches the current token in the word, and then increments that transition's weight by 1.
    
    Each time a string ends in a state, that state's `finalweight` attribute is also incremented by 1.
    """
    for word in data_iterator(path):
        cs = dfa.initialstate # gets set back to q0 when a new word is processed
        for token in word:
            for _, trans in cs.all_transitions():
                
                if trans.label == token: # find the transition that matches the current token
                    trans.weight += 1
                    cs = trans.targetstate # update cs to transition's target state. This is how we "traverse" the states
                    
        cs.finalweight += 1 # update the weight of the last state the current string ends in
    return dfa

def normalize_SL(dfa:FST) -> FST:
    pass


## Strictly 2-Piecewise Functions

In [125]:
def make_2_SP_dfa(alphabet:list) -> list[FST]:
    """This function intitializes 2SP machines according the length of the alphabet"""
    _2_SP_machines = []
    
    # initializes all 2 state machines and loopify them
    for sym1 in alphabet:
        dfa             = FST()
        q0              = dfa.initialstate
        q0.finalweight  = 0
        q0.name         = Lfish
        q1              = State()
        q1.finalweight  = 0
        q1.name         = sym1
        dfa.states      = {q0, q1}
        dfa.finalstates = {q0, q1}
        dfa.alphabet    = alphabet
        q0.add_transition(q1, sym1, 0)
        
        # loopification
        for sym2 in alphabet:
            q1.add_transition(q1, sym2, 0)
        
        _2_SP_machines.append(dfa)
     
    return _2_SP_machines
    


        
def update_SP(dfas:list[FST], path:str) -> list[FST]:
    pass

def normalize_SP(dfas:list[FST]) -> list[FST]:
    pass


## MLE Calculators (relative frequency)

In [ ]:
def MLE(dfa, path) -> FST:
    """This function takes a weighted DFA and normalizes its weights"""
    dfa = update_SL(dfa, path)
    
    # sums all the transition weights (counts) and the finalweight for given state 
    get_trans_sum = lambda state: sum([tran.weight for _, tran in state.all_transitions()])
    
    # gets the sum of all transitions and finalweight for each state.
    all_state_trans_sums = [get_trans_sum(state) + state.finalweight for state in dfa.states]
    
    for i, state in enumerate(dfa.states):
        for _, trans in state.all_transitions():
            
            #  normalize transition weight by sum of the state's transition weights
            trans.weight = (trans.weight) / all_state_trans_sums[i]
            
        state.finalweight = (state.finalweight / all_state_trans_sums[i])

    # assert that each state's transition weights sum to 1
    for state in dfa.states:
        assert sum([t.weight for _, t in state.all_transitions()]) + state.finalweight == 1
    
    return dfa

## Playground

In [69]:
# toy_alphabet = make_alphabet("data/LearningData.txt")
# toy_2SL = make_2_SL_dfa(toy_alphabet)
# toy_MLE = MLE(toy_2SL, "data/LearningData.txt")

toy_alphabet = make_alphabet("data/eric_debugging_data.txt")

toy_2SL = make_2_SL_dfa(toy_alphabet)
toy_MLE = MLE(toy_2SL, "data/eric_debugging_data.txt")


toy_2SP = make_2_SP_dfa(toy_alphabet)

86

##  notes

## ASK ABOUT THE EVALUATION STEP

MLE(dfa, corpus) = dfa' 

create_SL2_dfa (alphabet

getalphabet(corpus) = set_of_letters 

Q = all strings of length less than k

I = "" 

F = Q

delta(q,a) = k-1 suffix of qa 

example: (a,b) = k-1 suffix of ab which is b (k=2)

Example; Suppose k=4. Then delta(aaa,b) = 3 suffix of aaab, which is aab 

Also if qa is of length less than k-1 than the k-1 suffix of qa is just qa 

3 suffix of aa is aa

BIGRAM: all possible states given alphabet (given k)

42 SP machines, 1 SL machine

**NOTE:**
Multi-symbol labels get delimited by colons when displayed via `render`. So, "abc" -> "a:b:c" (for some really odd reason)